In [17]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

Load and scale the dateset

In [37]:
data = pd.read_csv('/home/kxlu/Documents/xai_compare/data/time/ETTh1.csv').drop('date',axis=1)
scalers={}
for i in data.columns:
    scaler = MinMaxScaler(feature_range=(-1,1))
    s_s = scaler.fit_transform(data[i].values.reshape(-1,1))
    s_s=np.reshape(s_s,len(s_s))
    scalers['scaler_'+ i] = scaler
    data[i]=s_s
train_df = data.iloc[:int(data.shape[0]*0.8)]
test_df = data.iloc[train_df.shape[0]:]

Transform the dataset into training samples using rolling window

In [6]:
def split_series(series, n_past, n_future):
  
  X, y = list(), list()
  for window_start in range(len(series)):
    past_end = window_start + n_past
    future_end = past_end + n_future
    if future_end > len(series):
      break
    past, future = series[window_start:past_end, :], series[past_end:future_end, :]
    X.append(past)
    y.append(future)
  return np.array(X), np.array(y)

In [38]:
lag =320; horizon = 32
X_train,y_train = split_series(train_df.values, n_past=lag, n_future=horizon)
X_test,y_test = split_series(test_df.values, n_past=lag, n_future=horizon)
n_features = X_train.shape[-1]

### LSTM

In [32]:
encoder_inputs = tf.keras.layers.Input(shape=(lag, n_features))
encoder = tf.keras.layers.LSTM(100, return_state=True)
encoder_outputs1 = encoder(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

decoder_inputs = tf.keras.layers.RepeatVector(horizon)(encoder_outputs1[0])

decoder = tf.keras.layers.LSTM(100, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(n_features))(decoder)

model = tf.keras.models.Model(encoder_inputs,decoder_outputs)

model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 320, 7)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 100),     │     43,200 │ input_layer_2[0]… │
│                     │ (None, 100),      │            │                   │
│                     │ (None, 100)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 32, 100)   │          0 │ lstm_4[0][0]      │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ (None, 32, 100)   │     80,400 │ repeat_vector[0]… │
│                     │                   │            │ lstm_4[0][1],     │
│                     │                   │            │ lstm_4[0][2]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed    │ (None, 32, 7)     │        707 │ lstm_5[0][0]      │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 124,307 (485.57 KB)

 Trainable params: 124,307 (485.57 KB)

 Non-trainable params: 0 (0.00 B)

In [39]:
model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())
history=model.fit(X_train,y_train,epochs=5,batch_size=32,verbose=0)

In [73]:
def split_series_X(series, n_past):
    X = list()
    for window_start in range(len(series)):
        past_end = window_start + n_past
        if past_end >= len(series):
            break
        past = series[window_start:past_end, :]
        X.append(past)
    return np.stack(X)

def preprocess(ts):
    ts = ts.to_numpy()
    X = split_series_X(ts, n_past=lag)
    return X

In [74]:
preprocess(train_df.iloc[:1000]).shape

(680, 320, 7)

In [76]:
from omnixai.data.timeseries import Timeseries
from omnixai.explainers.timeseries.auto import TimeseriesExplainer
# Initialize a TimeseriesExplainer
explainers = TimeseriesExplainer(
    explainers=["shap", "mace"],        # Apply SHAP and MACE explainers
    mode="forecasting",                 # A  forecasting task
    data=Timeseries.from_pd(train_df.iloc[:1000]),  # Set data for initializing the explainers
    model=model,                     # Set the black-box anomaly detector
    preprocess=preprocess,
    postprocess=None,
    params={"mace": {"threshold": 0.001}}   # Additional parameters for MACE
)
# Generate local explanations
test_instance = Timeseries.from_pd(test_df[:1000])
local_explanations = explainers.explain(test_instance)

AssertionError: Explainer shap -- `ts_length` should be less than the length of the training time series